# Load a tracked particle and movie to see if locating workd

### Written by Matan 20230115

In [1]:
#%matplotlib inline
%matplotlib widget


In [2]:
#############imports######################################################################
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact


In [3]:
################functions1##################################################################
def l2n(x):
    return np.array(x)
def parseCircles(circles):
    x = []
    y = []
    r = []
    for i in range(len(circles[0])):
        x.append(circles[0][i,0])
        y.append(circles[0][i,1])
        r.append(circles[0][i,2])
        
    x,y,r = l2n(x), l2n(y),l2n(r)
    return x,y,r

In [4]:
#########################functions2#########################################################
def preProcessFrame(frame):
    #blur and grayout a frame
    
    # Convert to grayscale.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      
    # Convert to HSV and extract the saturation channel
    #sat = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)[:,:,0]

    # median filter
    #median = cv2.medianBlur(gray, 3)
    #gray_blurred = median
    # Blur using 3 * 3 kernel.
    blurSize = 10#5
    gray_blurred = cv2.blur(gray, (blurSize, blurSize))
    
    return gray_blurred

def detectCircles(frame,minRadius,maxRadius,param1=30,param2=25):
    
    minDist = minRadius-1
    dp = 0.5
    # Apply Hough transform on the blurred image.
    circles = cv2.HoughCircles(frame, 
                cv2.HOUGH_GRADIENT,dp, minDist, param1 = param1,
                param2 = param2, minRadius = minRadius, maxRadius = maxRadius)

    return circles
    
def captureFrame(video,frame_id):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = video.read()
    if ret:
        return frame
    else:
        return ret

def drawCircles(circles,ax ):
    if circles is not None:  
    # Convert the circle parameters a, b and r to integers.
    #detected_circles = np.uint16(np.around(detected_circles))
    #fig, ax = plt.subplots() 
        for pt in circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]
            circle = plt.Circle((a, b), r, color='r',fill=False)
            ax.add_patch(circle)
            #ax.imshow(frame, cmap = 'gray')

def drawCirclesDf(df,ax ):
    
    
    for index, row in df.iterrows():
        x,y,r = row.x,row.y,row.r
        circle = plt.Circle((x, y), r, color='r',fill=False)
        ax.add_patch(circle)

def circlesToDataFrame(circles,frameNum):    
    x,y,r = parseCircles(circles)
    data = {'frame':[int(frameNum)]*len(circles),'x':x,'y':y,'r':r}
    dft = pd.DataFrame(data = data)
    
    return dft


def cropImage(img,x0,y0,width,height):
    return img[y0:y0+height,x0:x0+width,:]
'''def cropImage(img,x0,y0,width,height):
    d = len(img.shape)
    if d==2: #grayscale
        return img[y0:y0+height,x0:x0+width]
    elif d==3: #rgb
        return img[y0:y0+height,x0:x0+width,:]
'''

'def cropImage(img,x0,y0,width,height):\n    d = len(img.shape)\n    if d==2: #grayscale\n        return img[y0:y0+height,x0:x0+width]\n    elif d==3: #rgb\n        return img[y0:y0+height,x0:x0+width,:]\n'

In [27]:
expSumFileName = 'C:\\Users\\User\\Documents\\Eden\locate_git_code\\packaged\\botsTracking\\expSumPack.csv' #'transportExpSum20230309bots21.csv'
expSum = pd.read_csv(expSumFileName)#,index_col=0)

In [28]:
#########################video_filenames###############################################
#fileName = "D:\Eden\20221208\fronters22\C0486shortLabeled.MP4"

#VERY GOOD

arenaSizePixels = 1090 # [pixels]
arenaSizeCm = 150 # [cm]



### Fetch Experiments details from experiment summary table

In [29]:
expSum[expSum.bots==53]#.iloc[82]

,Unnamed: 0,date,operator,bottype,stars,bots,passiveDiameter[cm],path,locatingFileName,minCircleRadius,...,comments,backedup,motorVal,production,linkingFileName,botLocatingFileName,botLinkingFileName,botsMiniVideoFileNamePack,botsLocatingFileNamePack,botsLinkingFileNamePack
36,78,20221218,Matan,aligner,1.0,53,28,D:\Matan\20221218\aligners53passive28cm\C0510.MP4,D:\Matan\20221218\aligners53passive28cm\C0510....,102,...,NaN,1.0,130,NaN,D:\Matan\20221218\aligners53passive28cm\C0510....,D:\Matan\20221218\aligners53passive28cm\C0510....,D:\Matan\20221218\aligners53passive28cm\C0510....,bots_aligner_N053_D028_C0510.MP4,bots_aligner_N053_D028_C0510.MP4_botsLoc.csv,bots_aligner_N053_D028_C0510.MP4_botsLink.csv
37,79,20221218,Matan,aligner,3.0,53,28,D:\Matan\20221218\aligners53passive28cm\C0511.MP4,D:\Matan\20221218\aligners53passive28cm\C0511....,102,...,first few minutes show scatter well,1.0,130,D:\Matan\production\C5011coppedTrimmedGray.MP4,D:\Matan\20221218\aligners53passive28cm\C0511....,D:\Matan\20221218\aligners53passive28cm\C0511....,D:\Matan\20221218\aligners53passive28cm\C0511....,bots_aligner_N053_D028_C0511.MP4,bots_aligner_N053_D028_C0511.MP4_botsLoc.csv,bots_aligner_N053_D028_C0511.MP4_botsLink.csv
38,80,20221218,Matan,aligner,0.0,53,28,D:\Matan\20221218\aligners53passive28cm\C0512.MP4,D:\Matan\20221218\aligners53passive28cm\C0512....,102,...,NaN,1.0,130,NaN,D:\Matan\20221218\aligners53passive28cm\C0512....,D:\Matan\20221218\aligners53passive28cm\C0512....,D:\Matan\20221218\aligners53passive28cm\C0512....,bots_aligner_N053_D028_C0512.MP4,bots_aligner_N053_D028_C0512.MP4_botsLoc.csv,bots_aligner_N053_D028_C0512.MP4_botsLink.csv
39,81,20221218,Matan,aligner,0.0,53,28,D:\Matan\20221218\aligners53passive28cm\C0513.MP4,D:\Matan\20221218\aligners53passive28cm\C0513....,102,...,NaN,1.0,130,NaN,D:\Matan\20221218\aligners53passive28cm\C0513....,D:\Matan\20221218\aligners53passive28cm\C0513....,D:\Matan\20221218\aligners53passive28cm\C0513....,bots_aligner_N053_D028_C0513.MP4,bots_aligner_N053_D028_C0513.MP4_botsLoc.csv,bots_aligner_N053_D028_C0513.MP4_botsLink.csv
40,82,20221217,Matan,fronter,3.0,53,28,D:\Matan\20221217\fronters53passive28cm\C0505.MP4,D:\Matan\20221217\fronters53passive28cm\C0505....,102,...,2 mins to edge,1.0,130,D:\Matan\production\C0505coppedTrimmedGray.MP4,D:\Matan\20221217\fronters53passive28cm\C0505....,D:\Matan\20221217\fronters53passive28cm\C0505....,D:\Matan\20221217\fronters53passive28cm\C0505....,bots_fronter_N053_D028_C0505.MP4,bots_fronter_N053_D028_C0505.MP4_botsLoc.csv,bots_fronter_N053_D028_C0505.MP4_botsLink.csv
41,83,20221217,Matan,fronter,1.0,53,28,D:\Matan\20221217\fronters53passive28cm\C0506.MP4,D:\Matan\20221217\fronters53passive28cm\C0506....,102,...,NaN,1.0,130,NaN,D:\Matan\20221217\fronters53passive28cm\C0506....,D:\Matan\20221217\fronters53passive28cm\C0506....,D:\Matan\20221217\fronters53passive28cm\C0506....,bots_fronter_N053_D028_C0506.MP4,bots_fronter_N053_D028_C0506.MP4_botsLoc.csv,bots_fronter_N053_D028_C0506.MP4_botsLink.csv
42,84,20221217,Matan,fronter,1.0,53,28,D:\Matan\20221217\fronters53passive28cm\C0507.MP4,D:\Matan\20221217\fronters53passive28cm\C0507....,102,...,NaN,1.0,130,NaN,D:\Matan\20221217\fronters53passive28cm\C0507....,D:\Matan\20221217\fronters53passive28cm\C0507....,D:\Matan\20221217\fronters53passive28cm\C0507....,bots_fronter_N053_D028_C0507.MP4,bots_fronter_N053_D028_C0507.MP4_botsLoc.csv,bots_fronter_N053_D028_C0507.MP4_botsLink.csv
43,85,20221217,Matan,fronter,0.0,53,28,D:\Matan\20221217\fronters53passive28cm\C0509.MP4,D:\Matan\20221217\fronters53passive28cm\C0509....,102,...,NaN,1.0,130,NaN,D:\Matan\20221217\fronters53passive28cm\C0509....,D:\Matan\20221217\fronters53passive28cm\C0509....,D:\Matan\20221217\fronters53passive28cm\C0509....,bots_fronter_N053_D028_C0509.MP4,bots_fronter_N053_D028_C0509.MP4_botsLoc.csv,bots_fronter_N053_D028_C0509.MP4_botsLink.csv


In [30]:
#Choose experiment to test
expNum = 40

In [31]:
row = expSum.iloc[expNum]

passiveDiameter = row['passiveDiameter[cm]']
fileNameLoc = row.botLocatingFileName
fileNameLinked = row.botLinkingFileName#'temporaryLinkingFile.csv'#row.linkingFileName

fileNameVideo = row.path

In [32]:
row

Unnamed: 0                                                                  82
date                                                                  20221217
operator                                                                 Matan
bottype                                                                fronter
stars                                                                      3.0
bots                                                                        53
passiveDiameter[cm]                                                         28
path                         D:\Matan\20221217\fronters53passive28cm\C0505.MP4
locatingFileName             D:\Matan\20221217\fronters53passive28cm\C0505....
minCircleRadius                                                            102
maxCircleRadius                                                            106
fps                                                                         25
cpp                                                 

In [33]:
fileNameLinked

'D:\\Matan\\20221217\\fronters53passive28cm\\C0505.MP4_botsLocatedFull20230329.csv_linked.csv'

In [34]:
fileNameLoc

'D:\\Matan\\20221217\\fronters53passive28cm\\C0505.MP4_botsLocatedFull20230402.csv'

In [35]:
# Load video file
video = cv2.VideoCapture(fileNameVideo)
# Load locating file
t = pd.read_csv(fileNameLinked)
df = pd.read_csv(fileNameLoc)
#get videos properties
width  = video.get(cv2.CAP_PROP_FRAME_WIDTH )
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT )
fps = video.get(cv2.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
frameCount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
pixelSize = arenaSizeCm/arenaSizePixels

## Locating params 

In [36]:
plt.ioff()
fig1 = plt.figure()
ax1 = fig1.gca()
#ax.imshow(frame,cmap='Greys')
plt.ion()
#fig1.canvas

In [37]:
xC,yC = 657, 544
xP,yP = 192,801
rA = ((xP-xC)**2+(yP-yC)**2)**0.5

In [40]:
########################################widget_cell#####################################################################################
#fig,ax = plt.subplots()
starting_point=1
frame_step=50
maxFrame = frameCount#20000#frameCount
trailLength = 5000

frameRange= range(starting_point,maxFrame,frame_step)
@interact(frameNum=frameRange)#df.frame.values)
def drawPassiveParticleOnFrame(frameNum):
    
    dff =df[df.frame==frameNum]
    
    x,y,r = dff.x.values,dff.y.values,dff.r.values
    
    circles = [[x,y,r]]
    img  = captureFrame(video,frameNum)
    imgCropped =cropImage(img,300,0,1200,int(height))
    
    #filter out false bots out of arena
    cond = ((dff.x-xC)**2+(dff.y-yC)**2)**0.5<rA/2
    dff = dff[cond]
    
    ax1.cla()
    ax1.imshow(imgCropped)
    
    
    drawCirclesDf(dff,ax1)
    cond = (frameNum-trailLength<t.frame) &(t.frame<frameNum)
    tC = t[cond]
    if len(tC)==0 :
        tC = t[t.frame==frameNum]
    
    #filter by frame
    cond = df.frame==frameNum
    dfC = df[cond]
    
    
    tp.plot_traj(tC,ax=ax1,label=True)
    drawCirclesDf(dfC,ax1)
    #ax.imshow(img, cmap='Greys')
    
HBox([fig1.canvas])

interactive(children=(Dropdown(description='frameNum', options=(1, 51, 101, 151, 201, 251, 301, 351, 401, 451,…

In [23]:
for index, row in expSum[expSum.index>16].iterrows():
    print(index, end=', ')

17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 

In [89]:
for index,row in expSum.iterrows():
    print(row.botLinkingFileName)

D:\Eden\29_12_22\aligners_21_7cm\C0548.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\aligners_21_7cm\C0547.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\aligners_21_7cm\C0544.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\aligners_21_7cm\C0542.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\frontes_21_7cm\C0541.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\frontes_21_7cm\C0540.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\frontes_21_7cm\C0539.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\29_12_22\frontes_21_7cm\C0538.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\27_12_22\fronters-12cm\C0537.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\27_12_22\fronters-12cm\C0536.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\27_12_22\fronters-12cm\C0535.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\27_12_22\fronters-12cm\C0534.MP4botLocatedFull20230315.csv_linked.csv
D:\Eden\27_12_22\fronters-12cm\C0533.MP4botLocatedFu